# Text Repräsentationen
In diesem Teil des Labors wird es um eine Einfürung in Text Repräsentationen gehen. 

Wie Ihr vermutlich schon wisst, repräsentieren Computer Daten in Binärcode. Um Euch das Problem, das Computer mit Texten haben aufzuzeigen, hat euch der Computer eine Nachricht hinterlassen, die ihr Entschlüsseln müsst... 

In [1]:
binary_message = "01001001 00100000 01110011 01110000 01100101 01100001 01101011 00100000 01100010 01101001 01101110 01100001 01110010 01111001 00100000 01101111 01101110 01101100 01111001 00100001 00001010 01001001 00100000 01101100 01101001 01101011 01100101 00100000 01101110 01110101 01101101 01100010 01100101 01110010 01110011 00101100 00100000 01101110 01101111 01110100 00100000 01110100 01100101 01111000 01110100 00101110"

In [2]:
binary_chars = binary_message.split()
message = ""
for binary_char in binary_chars:
    binary_char_integer = int(binary_char, 2)  # TODO
    character = chr(binary_char_integer)
    message += character
print(message)


I speak binary only!
I like numbers, not text.


Um Maschinen Text verständlich zu machen, müssen wir die Texte in eine andere Repräsentation bringen. Eine Kurze Einführung bieten die beiden folgenden Warm Up Aufgaben.

## Tokenization
Eine der ersten elementaren Schritte, um eine natürliche Sprache mit einem Computer zu verarbeiten ist das Aufteilen eines Fließtextes in kleinere Teile. Meist werden Texte direkt in sogenannte Tokens geteilt (Tokenization). Ein Token ist oftmals ein einzelnes Wort, es kann aber auch aus mehreren Wörtern bestehen, wenn diese zusammen für eine Entität stehen, z.B. die Stadt „New York“. Neben der Aufteilung in Tokens ist es auch möglich, einen Text zuerst in Sätze zu splitten (Sentence Tokenization). Welche Kombination der Methoden angewendet wird, hängt von der Aufgabe ab. 

### Warm Up 1: Tokenizer mit Regulären Ausdrücken

Die erste kleine Aufgabe besteht darin mit einem regulären Ausdruck ein Wort-Tokenizer zu bauen. (Diese Aufgabe sollte nicht mehr als 5 min. dauern) Der Tokenizer muss auch nicht perfekt sein. 

> Eine kurze Auffrischung über Reguläre Ausdrücke findet sich in https://regexr.com/

In [3]:
import re

text = "Die Ente lacht und quakt."

tokenizer_regex = re.compile(r"\s+")  # TODO
tokenizer_regex.split(text)


['Die', 'Ente', 'lacht', 'und', 'quakt.']

Das Grundprinzip eines Tokenizers sollte euch nun klar sein. In der Praxis empfiehlt es sich allerdings, auf bestehende Tokenizer-Implementierungen zurückzugreifen.
Im Python-Umfeld ist https://spacy.io/api/tokenizer oder https://www.nltk.org/api/nltk.tokenize.html sehr empfehlenswert.

# Encoding

Der nächste Schritt, um mit Texten Maschinelles Lernen zu betreiben, ist das Umwandeln der Wörter in eine numerissche Repräsentation. In der nächsten Warm-Up Aufgabe wird es darum um Bag of Words gehen.

## Warm Up 2: Bag of Words mit scikit-learn
Ein Ansatz um Texte bzw. Tokens als Zahlen zu represäntieren ist ein Bag of Words. Hier wird jeder Text als Vektor dargestellt (Länge: Länge des Vokabulars). Jeder Eintrag im Vektor steht für ein Wort im Vokabular.

In der nächsten Aufgabe wollen wir mit dem vorgegebenen Textkorpus, den Satz **"Die Ente singt und quakt."** in einen bag-of-words-encoded Vector umwandeln.
Dazu bietet sich der `CountVectorizer` an (siehe [Dokumentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)).

> Kleiner Tipp: Der `CountVectorizer` übernimmt diesmal die Tokenisierung.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ["Die Ente lacht und quakt.", "Die Ente singt und tanzt."]
vectorizer = CountVectorizer()

vectorizer.fit_transform(corpus)  # TODO
bag_of_words_encoded_sentence = vectorizer.get_feature_names_out()  # TODO
print(bag_of_words_encoded_sentence)


['die' 'ente' 'lacht' 'quakt' 'singt' 'tanzt' 'und']


Im  nächsten Schritt gebt die Länge der Satzrepräsentation aus. 

Wie verändert sich die Länge, wenn sich der Textkorpus vergrößert. Was sind die Vorteile und Nachteile von Bag of Words?

In [5]:
size_of_sentence_encoded=len(bag_of_words_encoded_sentence) #TODO
print(size_of_sentence_encoded)

7


> **_Die Länge ändert sich nur wenn der Textkorpus bisher unbekannte Wörter enthält, für die es noch keine Token gibt._**

## Word Embeddings

Natürliche Sprachen bieten uns mannigfaltige Möglichkeiten, dieselben Inhalte auf unterschiedliche Art und Weise auszudrücken. Wenn wir mit Texten arbeiten, reicht es daher in der Regel nicht aus, nur die Anzahl und das Vorkommen von bestimmten Wörtern in Texten zu betrachten, weil neben des bloßen Vorkommen eines Wortes auch dessen Anordnung und dessen Bedeutung eine Rolle spielt. Als zusätzliche und wichtige Ebene kommt hier also die Semantik ins Spiel.

Die Abbildung von Wörtern auf Vektoren erlaubt es uns, mit diesen zu rechnen und zum Beispiel Distanzen oder Ähnlichkeiten zu bestimmen. Embeddings haben den Vorteil, dass sie eine Dimensionsreduktion mit sich bringen und semantische Embeddings sorgen darüber hinaus dafür, dass "verwandte" Wörter einen geringen Abstand voneinander haben.

Wir wollen uns im Folgenden zunächst mit Word2Vec beschäftigen und eine einfache Version des CBOW-Ansatzes selbst implementieren.

Danach schauen wir uns Gensim als Bibliothek für Word-Embedding-Modelle an und werfen einen Blick auf Evaluationsmethoden für Embeddings sowie die ihnen inhärenten Biase.

## Aufgabe 1: Word2Vec CBOW
> You shall know a word by the company it keeps.
>
> -- <cite>J. R. Firth</cite>

Auf dem oben zitierten Prinzip beruhen die beiden als Word2Vec bekannt gewordenen Modelle CBOW und Skip Gram, die 2013 von [Tomas Mikolov et al.](https://arxiv.org/abs/1301.3781) bei Google entwickelt wurden.
Erstgenanntes Modell werden wir im Folgenden in einer einfachen Form selbst implementieren.

#### 1.1 Trainingsdaten
Die Beschaffung und Aufbereitung von Trainingsdaten ist ein wichtiger Schritt in jeder NLP-Pipeline. Jetzt drücken wir uns mal davor und greifen auf einen Datensatz zu, den wir schonmal vorarb für Euch vorbereitet haben. Wir haben uns entschieden Alice im Wunderland [Gutenberg Project Alice im Wunderland](https://www.gutenberg.org/cache/epub/19778/pg19778.txt) zu nutzen um Word Embeddings zu trainieren. Der Datensatz beinhaltet die Sätze aus aus der Geschichte. 

Der vorverarbeitete Datensatz ist als pickle abgespeichert und findet sich in [hier](https://drive.google.com/file/d/1RfCivF-wHf33S7TMxjpT78923ADXreMi/view). Wir werden den googledrivedownloader nutzen, um die Datei zu laden. Ladet den Datensatz mit Pickle als Testdatensatz aus.

In [6]:
import gdown

gdown.download(
    id="1RfCivF-wHf33S7TMxjpT78923ADXreMi", output="./download/alice_sentences.pkl", 
)


Downloading...
From: https://drive.google.com/uc?id=1RfCivF-wHf33S7TMxjpT78923ADXreMi
To: /Users/sschwarzer/Desktop/repositories/ai-lab/nlp/download/alice_sentences.pkl
100%|██████████| 219k/219k [00:00<00:00, 7.85MB/s]


'./download/alice_sentences.pkl'

In [7]:
import pickle

pkl = open("./download/alice_sentences.pkl", "rb")
data_sample = pickle.load(pkl) # pickle riiiick
pkl.close()


### 1.2 Datenvorbereitung
Wir haben einen tokenisierten Datensatz, der aus Listen von Wörtern besteht. Jede Liste repräsentiert einen Satz. Unser Modell soll aber hinterher mit Zahlen hantieren und zwar entweder mit Wortindizes, die jedes Wort im Vokabular über einen eindeutige Nummer referenzierbar machen, oder mit One-hot-Vektoren, die als Labels dienen, mit denen der tatsächliche Output des Modells verglichen werden kann.

In [8]:
# from collections import OrderedDict # FIX: Not required after Python 3.7

# Beim Mapping von Wörtern zu IDs und umgekehrt sollte eine reproduzierbare Reihenfolge sichergestellt werden,
# um das Modell später weitertrainieren und die Embedding-Matrix interpretieren zu können.
# Diese Datenstruktur kann, aber muss nicht, als Basis dienen.
vocabulary = set([word for sentence in data_sample for word in sentence]) # ! This is added
unique_words = dict.fromkeys(vocabulary) # Liste alle Wörter, die in unserem data_sample vorkommen 

# Mapping von Wort zu ID
word2id = {word : index for index, word in enumerate(unique_words.keys())} # TODO

# Mapping von ID zu Wort
id2word = {index : word for index, word in enumerate(unique_words.keys())} # TODO

# Unser Data Sample, aber mit IDs statt Wörtern 
# [['der', 'hund', 'der', 'bellt'], ['die', 'katz', 'miaut']] => [[0, 1, 0, 2], [3, 4, 5]]
numeric_docs = [[word2id[w] for w in doc] for doc in data_sample]

print('Word to id sample:', list(word2id.items())[:10], '\n')
print('Id to word sample:', list(id2word.items())[:10], '\n')
print('Documents as lists of integers:', numeric_docs[0][:10])


Word to id sample: [('wisperte', 0), ('hübscher', 1), ('trinkt', 2), ('sauberes', 3), ('knurren', 4), ('Geschrei', 5), ('höre', 6), ('aufgraben', 7), ('herum.«', 8), ('zusammengezogen', 9)] 

Id to word sample: [(0, 'wisperte'), (1, 'hübscher'), (2, 'trinkt'), (3, 'sauberes'), (4, 'knurren'), (5, 'Geschrei'), (6, 'höre'), (7, 'aufgraben'), (8, 'herum.«'), (9, 'zusammengezogen')] 

Documents as lists of integers: [1922, 433, 620, 2702, 1597, 2620, 4149]


Wir halten einige wichtige Parameter für unser Modell fest. Die Größe des Kontextfensters sowie die Länge der Embeddingvektoren können nach Bedarf angepasst werden. Für unsere Demo wählen wir kleine Werte.

In [9]:
vocabulary_size = len(vocabulary)  # TODO
embedding_size = 50  # Länge der Embeddingvektoren
window_size = 2  # Größe des Kontextfensters. Wird nach rechts und links angewandt. Gesamter Kontext hier also 4 Wörter.

print("Vocabulary Size:", vocabulary_size)


Vocabulary Size: 4399


### 1.3 Generator
Um nicht alle Trainingsdaten auf einmal im Speicher halten zu müssen, schreiben wir uns eine Generatorfunktion, die Batches einer frei wählbaren Größe zurückgibt. Unser Ansatz ist dennoch nicht völlig speicherschonend, weil wir uns die Datengrundlage für die Generierung dieser Batches, nämlich die Integerlisten in `numeric_docs` sehr wohl im Speicher vorhalten. Darüber sehen wir aber großzügig hinweg.

Die Generatorfunktion erzeugt zwei numpy-Arrays der Länge `batch_size`, von denen das eine Array eine Liste mit Indizes der Kontexwörter enthält, die der Embedding-Layer als Eingabe erwartet, und das andere die zugehörigen One-Hot-Encodings der Mittelwörter.

Fiktives und vereinfachtes Beispiel:
<pre><code>* Fenstergröße: 1
* Batch-Size: 2
* Wortindizes: 'die': 0, 'ente': 1, 'lacht': 2, 'und': '3, 'quakt': 4 (und damit Vokabulargröße 5)
* Korpus (Auszug): [['die', 'ente', 'lacht', 'und', 'quakt'], ...]


=> Rückgabe:  
    contexts = [[0, 2], [1, 3]], 
    label_words = [[0, 1, 0, 0, 0], [0, 0, 1, 0, 0]]

=> Alternative Rückgabe:
    contexts = [['die', 'lacht'], ['ente', 'und']]
    label_words = [['ente'], ['lacht']]
</code></pre>

In [10]:
from keras.preprocessing import sequence
from keras.utils import np_utils
import numpy as np
from typing import Union, List, Iterator, Tuple

# contexts = 2 * window_size * batch_size
# label_words = batch_size * vocabulary_size

def generate_context_word_batches(
    corpus: Union[List[List[str]], List[List[int]]], # numeric_docs
    window_size: int,
    vocab_size: int, 
    batch_size: int, # definiert länge von contexts und label_words
) -> Iterator[Tuple[np.ndarray, np.ndarray]]:
    """
    Args:
        corpus: Tokenized text, split into list of strings for each sentence.
        window_size (int): Number of words watched to the left/right of each word (aka. context).
        vocab_size (int): Number of words in the vocabulary
        batch_size (int): Length of the two returned arrays

    Yields:
        List[List[int]]: Two arrays of size batch_size with the first 
    """
    word2id = {word: index for index, word in enumerate(set(word for sentence in corpus for word in sentence))} # word2id 
    current_size = 0 # batch counter
    while True:
        # TODO: Here be dragons
        ctx = []
        labels = []
        for sentence in corpus:
            if (len(sentence) < window_size * 2 + 1):
                continue
            for label_index in range(window_size, len(sentence) - window_size):
                ctx.append([word2id[sentence[label_index - i]] for i in range(-window_size, window_size + 1) if i != 0][::-1])
                labels.append(np_utils.to_categorical(word2id[sentence[label_index]], len(word2id)))
                current_size += 1
                if (current_size == batch_size):
                    current_size = 0
                    contexts, label_words = np.array(ctx), np.array(labels) # needs to be a numpy array to use the model.fit() function
                    ctx, labels = [], [] # reset memory
                    yield np.array(contexts), np.array(label_words)  # zwei numpy arrays



2022-11-08 12:48:09.279696: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Pseudocode

```python

for sentence in data_sample:
    for (i = window_size; i < len(sentence) - window_size; i++) # loop over label_words in sentence
        for word_index in range(-window_size, window_size) # loop over (left and right) contexts
            current_word = sentence[word_index]    
            if (word_index != 0): # if word is not label_word
                X.add(current_word)
            else:
                Y.add(current_word)
        current_size += 1
        if (current_size == batch_size):
            yield X, Y

    sentence_counter += 1




```

In [11]:
# Schneller Test
test_batch_size = 3
test_window_size = 2

batch_gen = generate_context_word_batches(corpus=numeric_docs, window_size=test_window_size, vocab_size=vocabulary_size, batch_size=test_batch_size)
for i in range(0, 3): 
    x, y = next(batch_gen)
    for j in range(0, test_batch_size):
        print('Context (X):', [id2word[w] for w in x[j]], '-> Target (Y):', id2word[np.argwhere(y[j])[0][0]])  

Context (X): ['Erstes', 'Kapitel', 'in', 'den'] -> Target (Y): Hinunter
Context (X): ['Kapitel', 'Hinunter', 'den', 'Kaninchenbau'] -> Target (Y): in
Context (X): ['Hinunter', 'in', 'Kaninchenbau', '.'] -> Target (Y): den
Context (X): ['Alice', 'fing', 'sich', 'zu'] -> Target (Y): an
Context (X): ['fing', 'an', 'zu', 'langweilen'] -> Target (Y): sich
Context (X): ['an', 'sich', 'langweilen', ';'] -> Target (Y): zu
Context (X): ['sich', 'zu', ';', 'sie'] -> Target (Y): langweilen
Context (X): ['zu', 'langweilen', 'sie', 'saß'] -> Target (Y): ;
Context (X): ['langweilen', ';', 'saß', 'schon'] -> Target (Y): sie


### 1.4 Definition des Models
Als nächstes definieren wir unser Model. Dazu verwenden wir die Sequential API von Keras. Dieser [Blogpost](https://lilianweng.github.io/lil-log/2017/10/15/learning-word-embedding.html) bietet nochmal eine anschauliche Erklärung, wie word2vec CBOW funktioniert.
Das folgende Bild ist daraus und stellt die Architektur des Neuronalen Netzes dar:  
<img src="https://lilianweng.github.io/lil-log/assets/images/word2vec-cbow.png" width=500 />

Weitere Informationen über den Embedding Layer finden sich [hier](https://keras.io/layers/embeddings/).

> **Tipp:** Nehmt euch wirklich Zeit Embeddings zu verstehen.

In [12]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

# ! https://github.com/nzw0301/keras-examples/blob/master/CBoW.ipynb

#Modelldefinition
cbow = Sequential()
# input_length resolves around the context length
cbow.add(Embedding(input_dim=vocabulary_size, output_dim=100, input_length=window_size*2)) #TODO
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_size,)))
# number of units equals every word out of the vocab, softmax to get a prob distribution
cbow.add(Dense(vocabulary_size, activation="softmax")) #TODO #Units #Activation
cbow.compile(loss="categorical_crossentropy", optimizer='rmsprop')

# Zusammenfassung
print(cbow.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            439900    
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4399)              444299    
                                                                 
Total params: 884,199
Trainable params: 884,199
Non-trainable params: 0
_________________________________________________________________
None


2022-11-08 12:48:16.213793: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 1.5 Training
Jetzt wird es ernst: Wir trainieren unser Modell.
Da wir eine eigene Generatorfunktion verwenden, müssen wir `steps_per_epoch` angeben. Überlegt euch was damit genau gemeint ist, wofür uns das nützt und was wir bei der Berechnung beachten müssen. Tipp: das Modell sollte pro Epoche alle Trainingsdaten sehen. Überlegt auch ob die Cbow-Fenstergröße einen einfluss auf diese Anzahl hat.

Weil wir unser Modell gerne abspeichern möchten, zum Beispiel, um es später weiter zu trainieren, definieren wir eine Callback-Funktion, die das für uns übernimmt.

In [13]:
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('embeddings.hd5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False)

In [14]:
epochs = 5  # (kann gerne erhöht werden)
batch_size = 300
# das Modell sollte pro Epoche alle Trainingsdaten sehen. Überlegt auch ob die Cbow-Fenstergröße einen einfluss auf diese Anzahl hat.
steps_per_epoch = sum(
    [max(len(sentence) - (window_size * 2 + 1), 0) for sentence in data_sample]
) // batch_size # TODO Dies ist die Anzahl der Generatoraufrufe pro Epoche. An sich ist es sinnvoll, diesen Wert auf ´#samples//batch_size`` zu setzen, damit das Modell pro Epoche alle Trainingsdaten sieht

# deprecated, .fit supports generators

# cbow.fit_generator(
#     generator=generate_context_word_batches(
#         corpus=data_sample,
#         window_size=window_size,
#         vocab_size=vocabulary_size,
#         batch_size=batch_size,
#     ),  # TODO
#     epochs=epochs,  # TODO
#     steps_per_epoch=steps_per_epoch,  # TODO
#     callbacks=[model_checkpoint],  # TODO
# )

# y and batch_size should not be specified when using a generator
history = cbow.fit(
    x=generate_context_word_batches(
        corpus=data_sample,
        window_size=window_size,
        vocab_size=vocabulary_size,
        batch_size=batch_size,
    ),
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    callbacks=[model_checkpoint],
)


Epoch 1/5
88/89 [============================>.] - ETA: 0s - loss: 8.1600
Epoch 1: loss improved from inf to 8.15636, saving model to embeddings.hd5
INFO:tensorflow:Assets written to: embeddings.hd5/assets
89/89 [==============================] - 3s 28ms/step - loss: 8.1564
Epoch 2/5
88/89 [============================>.] - ETA: 0s - loss: 7.2555
Epoch 2: loss improved from 8.15636 to 7.25068, saving model to embeddings.hd5
INFO:tensorflow:Assets written to: embeddings.hd5/assets
89/89 [==============================] - 3s 35ms/step - loss: 7.2507
Epoch 3/5
89/89 [==============================] - ETA: 0s - loss: 6.5115
Epoch 3: loss improved from 7.25068 to 6.51146, saving model to embeddings.hd5
INFO:tensorflow:Assets written to: embeddings.hd5/assets
89/89 [==============================] - 4s 44ms/step - loss: 6.5115
Epoch 4/5
89/89 [==============================] - ETA: 0s - loss: 6.3217
Epoch 4: loss improved from 6.51146 to 6.32169, saving model to embeddings.hd5
INFO:tensorflo

### 1.6 Test
Nachdem wir unser Modell nur sehr kurz und nur auf wenigen Daten trainiert haben, ist davon auszugehen, dass die Ergebnisse nicht optimal sind. Einen kurzen Blick wollen wir dennoch riskieren.

Dazu extrahieren wir zunächst die Gewichte aus dem Embedding-Layer und schauen sie uns auszugsweise an.

In [15]:
import pandas as pd
from keras.models import load_model

cbow = load_model("./embeddings.hd5") # TODO: Model laden
embedding_weights = cbow.get_layer("embedding").get_weights()[0] # TODO: Auf Embedding Layer (1. Layer) des Modells zugreifen und dort die Gewichtsmatrix extrahieren

pd.DataFrame(embedding_weights, index=list(id2word.values())).head() # TODO: Was wollen wir anschauen?

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
wisperte,0.019688,-0.020906,-0.010620,-0.015431,-0.033278,0.019629,-0.028774,-0.002464,-0.049655,0.055011,...,0.054802,0.048052,-0.002675,0.004507,-0.064263,-0.001790,0.048621,-0.060419,0.002684,-0.024934
hübscher,-0.031907,0.003123,0.045882,0.011462,-0.018991,0.062678,0.014064,-0.045243,-0.050458,-0.013482,...,-0.007344,-0.028524,-0.031760,-0.049186,0.010973,-0.002943,0.051939,-0.011931,-0.009198,0.043524
trinkt,-0.009499,0.014846,-0.006785,-0.020151,0.053994,0.013552,0.004127,-0.039954,-0.039460,-0.018305,...,0.031358,0.017762,0.047564,0.019043,0.014842,0.014977,0.063935,-0.040929,0.030067,0.046317
sauberes,0.014415,-0.033019,-0.016654,0.005282,-0.021218,0.004291,-0.019148,-0.019716,-0.035020,0.060449,...,0.059903,0.065154,-0.033325,-0.001673,0.039711,-0.013402,0.038106,-0.062765,0.040553,-0.024895
knurren,0.005091,0.007309,-0.033350,-0.003477,0.059276,0.056922,0.013418,-0.023101,-0.001393,-0.013006,...,-0.020489,0.024506,-0.024635,-0.064822,0.028294,0.005668,0.007854,-0.027124,0.060357,0.022493


Da durch scharfes Hinsehen nicht unmittelbar zu erkennen ist, wie gut unsere Embeddings schon sind, machen wir stichprobenartige Tests. Dazu wählen wir einige Wörter und berechnen für deren Embeddings die Ähnlichkeit mit allen anderen Embedding-Vektoren in unserer Gewichtsmatrix. Anschließend lassen wir uns die fünf ähnlichsten Wörter ausgeben.
Überlegt euch welches Distantzmaß für den Vergleich von Vektoren genutzt werden kann. Tipp: Die Cosinusähnlichkeit könnte damit was zu tun haben. 

In [16]:
from sklearn.metrics.pairwise import cosine_similarity # Tipp: from sklearn.metrics.pairwise import ?

sample_terms = ['Alice', 'Hut', 'Kaninchen','Kaninchenbau']
sample_embeddings = [embedding_weights[word2id[term]] for term in sample_terms] # TODO

# Berechne die paarweisen Distanzen zwischen Beispielwörtern und Gesamtvokabular
distance_matrix = cosine_similarity(sample_embeddings, embedding_weights)# TODO

In [17]:
# Zeige die top fünf ähnlichsten Wörter zu unseren Beispielwörtern 
similar_words = {sample_term: [id2word[idx] for idx in distance_matrix[index].argsort()[1:6]] 
                   for index, sample_term in enumerate(sample_terms)}

similar_words

{'Alice': ['gegeben.', 'zurückgekommen', 'Worauf', 'Woran', 'Mensch'],
 'Hut': ['Hierauf', 'Mensch', 'Gründe', 'zurückgekommen', 'verursachte'],
 'Kaninchen': ['gegeben.', 'schenken', 'Mensch', 'Woran', 'zurückgekommen'],
 'Kaninchenbau': ['gegeben.',
  'Mensch',
  'bekommt',
  'eleganten',
  'zurückgekommen']}

Seid ihr zufrieden mit den ähnlichen Wörtern? Woran kann es liegen, dass die Wörter nicht immer unbedingt Sinn ergeben?

## Aufgabe 2: Gensim als Wrapper für Word2Vec-Modelle
Embedding-Layer begegnen einem in der Praxis in der Tat häufig. In der Regel aber nicht als Bestandteile von reinen Word Embedding-Trainingsmodellen, sondern als erster Layer für Modelle mit anderen Aufgaben. Die Embeddings werden dann entweder mit vorberechneten Werten initalisiert und/oder werden im Training des Modells für die Downstream-Aufgabe (Textklassifikation, Übersetzung, ...) mittrainiert.

Eine komfortable Möglichkeit, eigene Word2Vec-Modelle zu trainieren, bietet [Gensim](https://radimrehurek.com/gensim/models/word2vec.html), eine Bibliothek, die für diese Modelle auch Wrapper bereitstellt, um komfortabler an die Embeddings zu kommen und mit diesen zu arbeiten.

Wir wollen uns im Folgenden einen kleinen Ausschnitt der Möglichkeiten, die Gensim bietet, anschauen.

### 2.0 Vorbereitung
Wir werden mit vortrainierten Google-News-Embeddings arbeiten, die ihr mit dem Code in den nächsten beiden Zellen herunterladen könnt. Falls das nicht funktionieren sollte, findet ihr die vortrainierten Embeddings [hier](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing). 

In [18]:
!pip install gdown

In [26]:
from gdown import download
import os.path

if not os.path.isfile("./download/GoogleNews-vectors-negative300.bin"): 
    download(
        id="1Fl11N_cX1RfJmTHV1C-RGIsjWeIZjbTN",
        output="./download/GoogleNews-vectors-negative300.bin.gz",
    )


### 2.1 Word2Vec-Model laden
Die Vektoren haben eine Länge von 300.
Zieht euch die Embeddings über den oben angegebenen Link und verwendet gensim, um sie anschließend zu laden.

**Hinweis**: Es kann einen Moment dauern, bis das Dictionary, das von Wort auf Embedding abbildet, erzeugt ist. Im Zweifel ein ```limit``` angeben und nur die ersten 1,5 Mio. Embeddings laden.

In [27]:
if not os.path.isfile("./download/GoogleNews-vectors-negative300.bin"): 
    !gunzip ./download/GoogleNews-vectors-negative300.bin.gz

In [28]:
import gensim

embeddings = gensim # TODO

SyntaxError: invalid syntax (1252212729.py, line 3)

Aus Spaß an der Freude können wir nun schauen, wie gut unser trainiertes Modell ist bzw. ob es bestimmte von Menschen wahrgenommene Analogien bestätigt. 

Die Analogien sind [hier](https://github.com/nicholas-leonard/word2vec/blob/master/questions-words.txt) zu finden.

In [ ]:
from gensim.test.utils import datapath

embeddings.evaluate_word_analogies(datapath("questions-words.txt"), restrict_vocab=30000)

### 2.2 Spaß mit Semantik
Im Folgenden wollen wir uns mit dem Mehrwert beschäftigen, den semantische Embeddings bieten. Für weitere Inspiration siehe zum Beispiel [hier](https://www.machinelearningplus.com/nlp/gensim-tutorial/) und die [Doku](https://radimrehurek.com/gensim/models/keyedvectors.html).

Mit semantischen Vektoren lassen sich zum Beispiel folgende Fragen beantworten:


In [ ]:
# Welche Stadt ist das New York Deutschlands? (Hinweis: 'New_York' ist als Token in den Embeddings enthalten)
print('Das deutsche New York ist: {}\n'.format(# TODO)))

# Was ist Emacs besonders ähnlich?
print('Ähnlichste Begriffe zu "Emacs": {}\n'.format(# TODO))

# Und wie sieht es mit Vim aus?
print('Ähnlichste Begriffe zu "Vim": {}\n'.format(# TODO))

# Wer ist eigentlich der Mozart der Naturwissenschaft?
print('Der Mozart der Naturwissenschaft ist: {}\n'.format(# TODO))

# Welches Wort verhält sich zu 'singing' wie 'burnt' zu 'burning'?
print('burning:burnt wie singing:{}\n'.format(# TODO))

# Sind sich Deutschland und Frankreich ähnlicher oder Deutschland und Kanada?
print('Ähnlichkeit DE, FR: {}'.format(# TODO))
print('Ähnlichkeit DE, CAN: {}'.format(# TODO))

Bei der Interpretation der Ergebnisse ist jedoch Vorsicht geboten: Es werden zwar semantische Beziehungen abgebildet, aber die entsprechen möglicherweise nicht immer den Erwartungen.

Wie ähnlich sind sich zum Beispiel "Leben" und "Tod", "kalt" und "warm", "Norden" und "Süden"?

Sind die Ergebnisse wie erwartet? Warum (nicht)?

In [ ]:
# TODO: Ähnlichkeiten berechnen

Die Embeddings sind nicht neutral, sondern spiegeln die Beziehungen wieder, die sich in den Trainingsdaten finden lassen.

In [ ]:
# Wird Wissenschaft von Frauen oder Männern gemacht?
print('Wissenschaft wird gemacht von: {}'.format(# TODO))
# Sind Mörder eher Schwarze, Weiße oder Asiaten?
print('Mörder sind: {}'.format(# TODO))
# Was bleibt vom Mann, wenn die Intelligenz abgezogen wird?
print('Mann ohne Intelligenz: {}'.format(# TODO))